In [5]:
import cv2 as cv
import numpy as np
import mysql.connector
import datetime



#Initialise Camera
vid = cv.VideoCapture(0)
dt=datetime.date.today()
#Connecting to the db
mydb=mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="",
    database="cisco_thingqbator"
    )
db_cursor1=mydb.cursor()

#Create a cascade classifier object
face_cascade = cv.CascadeClassifier("haarcascade_frontalface_default.xml")
faces=[]

#No. of frames
nof = 0

#List to store each face, to be done later
face_list=[]
path = "imgs/"

while True:
    #checks if the day hasn't changed or if the month has changed
    now=datetime.date.today()
    if dt.day+1==now.day or dt.day>now.day:
        dt=datetime.date.today()
        face_list=[]

    #check returns bool, frame returns the image
    check, frame = vid.read()
    if not check:
        continue
        
    #Increment number of frames
    nof += 1
    
    #Convert image to grayscale
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    
    #Detect multiple faces. Returns a list of faces in the current frame.
    faces = face_cascade.detectMultiScale(gray, 1.5, 5)
    
    if len(faces)==0:
        continue
    
    #Sort the faces in decreasing order on basis of features
    faces = sorted(faces, key=lambda x: x[2]*x[3], reverse=True)
    
    #Draw a rectangle around each face
    for top, left, bottom, right in faces:
        gray = cv.rectangle(gray, (top, left), (top+bottom, left+right), (0,255,0), 1)
        
    
    #Process only if faces is not empty
    if len(faces)!=0:
        for face in faces[:1]:
            x, y, w, h = face
            
            # Get the face ROI(region of interest)
            offset = 7
            face_section = gray[y-offset:y+h+offset, x-offset:x+w+offset]
            face_section = cv.resize(face_section, (100, 100))
                                
            #Append the ROI in the list after every 10 frames
            if nof % 10 == 0 :
                face_list.append(face_section)
        
        
    #Show the faces with rectangle drawn on them
    cv.imshow("Face", gray)
    
    #Wait for 1ms per frame. Quit if 'q' pressed.
    key = cv.waitKey(1)
    if key == ord('q'):
        break
    
#Print no. of frames
print(nof)
for i in face_list:
    print(i)
face_list = np.asarray(face_list)

for i in range(len(face_list)):
    np.save(path + str(i), face_list[i])
vid.release()
cv.destroyAllWindows()


17
[[67 70 72 ... 58 65 67]
 [70 72 73 ... 59 66 67]
 [67 65 65 ... 64 63 60]
 ...
 [29 28 28 ... 24 26 24]
 [27 27 28 ... 24 24 25]
 [28 27 28 ... 23 23 24]]
